In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re #regular expressions
import scipy.stats
import datetime as dt
%matplotlib inline
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn  import preprocessing
from sklearn.decomposition import PCA 
from sklearn import ensemble
from sklearn.model_selection import cross_val_score, GridSearchCV

#specifically nlp stuff

import re
from nltk.corpus import  stopwords
from collections import Counter
import nltk
from sklearn.model_selection import train_test_split

In [2]:
#import cleaned database
music = pd.read_csv(r'C:\Users\Sean\Documents\Thinkful\RAP NLP\CSV\music_clean_final.csv')

In [3]:
#needed to generate data for doc2vec
import csv
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import TaggedDocument
import spacy
import en_core_web_sm  # or any other model you downloaded via spacy download or pip
import gensim
from nltk.tokenize import word_tokenize
from gensim.models import word2vec
nlp = en_core_web_sm.load()
nltk.download('wordnet')

D:\Anaconda\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#convert to string
music.lyrics = music.lyrics.astype(str)
#parse data
lyrics = music.lyrics.apply(lambda x: nlp(x))

In [6]:
music['parsed_lyrics'] = lyrics
music['parsed_lyrics'].astype(str)

0       party people in the place. embrace the bass as...
1       here i am    raw. terrorist here to bring trou...
2       let it roll get bold i just cant hold. back or...
3       talk about ever meeting ms right. but one thin...
4       yo on the mic right about now i go by the name...
5       mmm mmm mmm. aw yea im with this. im just gonn...
6       hear ye hear ye from far and near. the one the...
7       funky . ready . funky funky ahhh  . one two on...
8        who is the man with the master plan . dj mist...
9       grab a hold of yourself open your eyes. get wi...
10       dr harrah is mr kane loose in this hospital ....
11      here comes the conquering brother that fathoms...
12      the kiss of death on a rap pick. then you get ...
13      on and on and on and on. me say the beat dont ...
14      so out of the mercy of allah. and the law writ...
15      well excuse me take a few minutes to mellow ou...
16       yo whassup big daddy .  aiyyo whassup red ale...
17      he has

# doc2vec

In [7]:
import nltk
music['tokenized_lyrics'] = music.apply(lambda row: nltk.word_tokenize(row['lyrics']), axis=1)

In [9]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re

lmtzr = WordNetLemmatizer()
w = re.compile("\w+",re.I)

In [10]:
def label_sentences(df):
    labeled_sentences = []
    for index, datapoint in df.iterrows():
        tokenized_words = re.findall(w,datapoint["lyrics"].lower())
        labeled_sentences.append(LabeledSentence(words=tokenized_words, tags=['SENT_%s' %index]))
    return labeled_sentences

In [11]:
def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(alpha=0.025, min_alpha=0.025, )
    model.build_vocab(labeled_sentences)
    for epoch in range(10):
        model.train(labeled_sentences, total_examples=1030, epochs=5)
        model.alpha -= 0.002 
        model.min_alpha = model.alpha
    
    return model

In [12]:
labelledlyrics = label_sentences(music)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [13]:
model = train_doc2vec_model(labelledlyrics)

In [14]:
def vectorize_comments(df,d2v_model):
    y = []
    comments = []
    for i in range(0,df.shape[0]):
        label = 'SENT_%s' %i
        comments.append(d2v_model.docvecs[label])
    df['vectorized_lyrics'] = comments
    
    return df

In [218]:
model.save('labelledlyrics.doc2vec')

In [236]:
model.wv.most_similar('sons')

[('daughters', 0.4074193239212036),
 ('fathers', 0.3978744149208069),
 ('mothers', 0.3895375430583954),
 ('dads', 0.3784688711166382),
 ('brothers', 0.3761492371559143),
 ('relationships', 0.36438578367233276),
 ('lanes', 0.33500170707702637),
 ('cousins', 0.3325534164905548),
 ('cheer', 0.3317328095436096),
 ('pride', 0.32994696497917175)]

In [16]:
#use the function defined above to get the vectors into the dataframe
df = vectorize_comments(music,model)
df = df.drop('parsed_lyrics', 1)
print (df.head(2))

                 name  duration_ms  popularity  num_markets  \
0  Long Live the Kane     295333.0        26.0         78.0   
1         Raw - Remix     344960.0        33.0         78.0   

                album  disc_number is_explicit  track_number release_date  \
0  Long Live The Kane          1.0       False           1.0         1988   
1  Long Live The Kane          1.0       False           2.0         1988   

           artist  ...  acousticness  instrumentalness  liveness  valence  \
0  Big Daddy Kane  ...        0.0556          0.000016     0.180    0.612   
1  Big Daddy Kane  ...        0.0186          0.000003     0.109    0.705   

     tempo  time_signature                                             lyrics  \
0   99.327             4.0  party people in the place. embrace the bass as...   
1  110.681             4.0  here i am    raw. terrorist here to bring trou...   

      region                                   tokenized_lyrics  \
0  eastcoast  [party, people, in, t

In [18]:
vectorlist = df["vectorized_lyrics"].T.tolist()

# Bag of Words

In [101]:
from collections import Counter
nltk.download('stopwords')

# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(25)]

def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [109]:
import random
#randomize dataset
music_random = music.sample(frac=1)
rapwords_df1 = music_random.iloc[0:330]
rapwords_df2 = music_random.iloc[331:660]
rapwords_df3 = music_random.iloc[661:1030]

#pick out some random words that are in common among songs
rapwords1 = rapwords_df1.iloc[np.random.choice(np.arange(len(rapwords_df1)), 300, False)]
rapwords2 = rapwords_df2.iloc[np.random.choice(np.arange(len(rapwords_df2)), 300, False)]
rapwords3 = rapwords_df3.iloc[np.random.choice(np.arange(len(rapwords_df3)), 300, False)]
# Set up the bags.
rapwords1 = rapwords1['parsed_lyrics'].apply(bag_of_words)
rapwords2 = rapwords2['parsed_lyrics'].apply(bag_of_words)
rapwords3 = rapwords3['parsed_lyrics'].apply(bag_of_words)
#extract words from lists in df to a series
rapwords1 = rapwords1.T.squeeze()
rapwords2 = rapwords2.T.squeeze()
rapwords3 = rapwords3.T.squeeze()

rapwords1 = rapwords1.apply(pd.Series).stack().reset_index(drop=True)
rapwords2 = rapwords2.apply(pd.Series).stack().reset_index(drop=True)
rapwords3 = rapwords3.apply(pd.Series).stack().reset_index(drop=True)
#need more data...consider including rare words..search for these functions in sklearn

In [111]:
#get the words into master lists
rapwords1 = rapwords1.tolist()
rapwords2 = rapwords2.tolist()
rapwords3 = rapwords3.tolist()

In [115]:
# return those commond words that can be found in all lists
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

intersection_1_2 = intersection(rapwords1, rapwords2)
intersection_1_2_3 = intersection(intersection_1_2, rapwords3)

In [154]:
def bow_feature_columns(bow_columns):
    bow_df = pd.DataFrame(columns=bow_columns)
    return bow_df

bow_df = bow_feature_columns(intersection_1_2_3)

#add song name and song lyric column
bow_df['song'] = music['name']
bow_df['lyrics'] = music['lyrics'].astype(str)


In [160]:
def letstry(words):
    for i, sentence in enumerate(bow_df['lyrics']):
        for word in words:
            bow_df.loc[i, word] = (bow_df['lyrics'].iloc[i]).count(word)

letstry(intersection_1_2_3)

In [161]:
print(bow_df)

     inside wish have feel live    d stick hoe flip feeling  ... dick will  \
0         0    1    2    0    4   75     0   0    0       0  ...    0    1   
1         0    0    0    0    0   64     0   0    0       0  ...    0    2   
2         0    1    1    1    0   78     1   0    1       0  ...    0    3   
3         1    0    1    5    1   95     0   0    0       3  ...    0    2   
4         0    0    1    0    0   97     0   0    0       0  ...    0    1   
5         0    0    4    0    0  129     0   0    0       0  ...    0    1   
6         0    1    0    0    4   69     0   1    0       0  ...    0    3   
7         0    0    0    0    0   79     0   0    0       0  ...    0    2   
8         0    0    0    1    1   74     0   0    0       0  ...    0    1   
9         0    0    0    0    0   57     0   0    0       0  ...    0    1   
10        0    0    1    0    2   96     0   0    0       0  ...    0    4   
11        0    0    2    0    0   94     1   0    0       0  ...

[1031 rows x 382 columns]


In [172]:
#feature selection
#bow_df = bow_df.drop('d',1)
bow_df['region'] = music['region']

In [194]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np


data = [go.Histogram(x=bow_df['region'])]

nbinsx=12

layout = go.Layout(
    title='Distribution of songs by region',
    xaxis=dict(
        title='Region'
    ),
    yaxis=dict(
        title='# of songs'
    ),
    bargap=0.1,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')


D:\Anaconda\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



# Trial model with BOW

In [211]:
X = bow_df.drop('region', 1)
X = X.drop('lyrics', 1)
Y = bow_df.region.astype(str)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [212]:
params = {'n_estimators': 200,
          'max_depth': 10,
          'loss': 'deviance'}

clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [213]:
predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

In [214]:
table_test = pd.crosstab(Y_test, predict_test, margins=True)

print(table_test)


col_0      eastcoast  midwest  south  westcoast  All
region                                              
eastcoast         87        6      4         16  113
midwest           10       27      1         12   50
south              7        4     25         16   52
westcoast         11        4      5         75   95
All              115       41     35        119  310


In [237]:
table_train = pd.crosstab(Y_train, predict_train, margins=True)
print(table_train)

col_0      eastcoast  midwest  nan  south  westcoast  All
region                                                   
eastcoast        266        0    1      0          0  267
midwest            0      136    0      0          0  136
nan                1        0    1      0          0    2
south              1        0    0     96          0   97
westcoast          0        0    0      0        219  219
All              268      136    2     96        219  721


In [ ]:
clf.feature_importance

# Trial Model with DOC2VEC

In [216]:
X = music['vectorized_lyrics']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)